In [113]:
import requests
import bs4 as bs
from bs4 import BeautifulSoup
import pandas as pd

response = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.X9DXmtj0mUl")
#print(response)
soup = BeautifulSoup(response.content, 'html.parser')
soup.prettify()

#soup.find_all('p')
outer = soup.find_all('p', class_='period-name')
str(outer)



'[<p class="period-name">Tonight<br/><br/></p>, <p class="period-name">Thursday<br/><br/></p>, <p class="period-name">Thursday<br/>Night</p>, <p class="period-name">Friday<br/><br/></p>, <p class="period-name">Friday<br/>Night</p>, <p class="period-name">Saturday<br/><br/></p>, <p class="period-name">Saturday<br/>Night</p>, <p class="period-name">Sunday<br/><br/></p>, <p class="period-name">Sunday<br/>Night</p>]'

In [171]:
#DAYS
days = []
for link in soup.find_all('p', class_='period-name'):  # iterate over every <a> tag
    days.append(link.get_text())
#print(days)

"""Need to seperate nights and days, so have just four days, 
 as only four values for other lists too (for DataFrame requirements):"""
night = [] 
daytime = [] 
for i in range(0, len(days)): 
    if i % 2: 
        daytime.append(days[i]) 
    else: 
        night.append(days[i])
print(daytime)
#print(night)    # Don't need to print because it will be discarded


#HIGH TEMPERATURES:
temp_f_high = [] 
for link in soup.find_all('p', class_="temp temp-high"):  # iterate over every <a> tag
    temp_f_high.append(link.get_text())
#print(temp_f_high)

temp_f_high[:] = [s.replace('High: ', '').replace('°F', '') for s in temp_f_high]
#print(temp_f_high)

#Temperature conversion F to C:
temp_c_high = []
for temp_f in temp_f_high:
    temp_c_high.append((int(temp_f)-32)*(5/9))


#LOW TEMPERATURES:
temp_f_low = []
for link in soup.find_all('p', class_="temp temp-low"):  # iterate over every <a> tag
    temp_f_low.append(link.get_text())
#print(temp_f_low)

temp_f_low[:] = [s.replace('Low: ', '').replace('°F', '') for s in temp_f_low]
#print(temp_f_low)

#Temperature conversion F to C:
temp_c_low = []
for temp_f in temp_f_low:
    temp_c_low.append((int(temp_f)-32)*(5/9))

"""Must make difference for timings - during night, there are 5 nights given,
during daytime there are 5 days given. Therefore arrays are different lengths."""
if len(temp_c_low) > len(temp_c_high):
    temp_c_low = temp_c_low[:-1]
elif len(temp_c_high) > len(temp_c_low):
    temp_c_high = temp_c_high[:-1]

#Final print to check lists:
print(temp_c_high)
print(temp_c_low)


#DESCRIPTION:
description = []
for link in soup.find_all('div', class_='row-forecast'):  # iterate over every <a> tag
    description.append(link.get_text())

night_desc = []
daytime_desc = []
for i in range(0, len(description)): 
    if i % 2: 
        daytime_desc.append(description[i]) 
    else: 
        night_desc.append(description[i])
daytime_desc = daytime_desc[:-3]
#print(daytime_desc)
#print(night_desc)    # Not needed


for string in daytime_desc:
    for day in daytime:
        string = string.replace(day, '')
    string = string.replace('Night', '')
    #print(string)
    names_list = [y for y in (x.strip() for x in string.splitlines()) if y]
    print(names_list)

#names_list[0].append(names_list[1])
#print(names_list)




['Thursday', 'Friday', 'Saturday', 'Sunday']
[16.11111111111111, 14.444444444444445, 15.0, 15.555555555555557]
[8.88888888888889, 7.222222222222222, 9.444444444444445, 9.444444444444445]
['Mostly sunny, with a high near 61. West wind 7 to 17 mph, with gusts as high as 22 mph.']
['Mostly sunny, with a high near 58. Calm wind.']
['A 30 percent chance of rain, mainly before 10am.  Mostly cloudy, with a high near 59. New precipitation amounts of less than a tenth of an inch possible.']
['A chance of rain, mainly before 4pm.  Mostly cloudy, with a high near 60.']


In [123]:
import pandas as pd
import numpy as np

"""df_days = pd.DataFrame({'Days': days})
print(df_days)

df_temp_low = pd.DataFrame({'Low Temp': temp_low})
print(df_temp_low)

df_temp_high = pd.DataFrame({'High Temp': temp_high})
print(df_temp_high)"""

df_description = pd.DataFrame({'Description': description})
print(df_description)

data = {'High Temp': temp_c_high,
        'Low Temp': temp_c_low}
labels = daytime

df = pd.DataFrame(data, index=labels)
print(df)


                                         Description
0  ThursdayMostly sunny, with a high near 61. Wes...
1  Thursday NightMostly clear, with a low around ...
2  FridayMostly sunny, with a high near 58. Calm ...
3  Friday NightA 30 percent chance of rain, mainl...
4  SaturdayA 30 percent chance of rain, mainly be...
5  Saturday NightA 20 percent chance of rain afte...
6  SundayA chance of rain, mainly before 4pm.  Mo...
          High Temp  Low Temp
Thursday  16.111111  8.888889
Friday    14.444444  7.222222
Saturday  15.000000  9.444444
Sunday    15.555556  9.444444
